In [85]:
import rasterio
import matplotlib.pyplot as plt
import numpy as np
import torch
import time
import os
import sys
sys.path.append('/home/esther/qr_for_landcover/scripts')
import landcover_definitions as lc
import util
import pickle
sys.path.append('../evaluation')
import run_model_forward_and_produce_tifs as run_model

In [86]:
torchgeo_pred_dir = '/home/esther/torchgeo_predictions_rep'
torchgeo_data_dir = '/datadrive/esther/torchgeo_data'
torchgeo_output_dir = '/home/esther/qr_for_landcover/output_rep'


In [87]:
classes_keep = [1,2,3,4,6]
ignore_index = len(classes_keep)

def reindex_ea(array_in, classes_keep):
    ignore_index = len(classes_keep)
    reindex_map = dict(zip(classes_keep, np.arange(len(classes_keep))))
    # reindex shrub to tree
    tree_idx = 3 # tree idx is 3 when there are no zeros
    shrub_idx = 5
    reindexed_mask = -1 * np.ones(array_in.shape)
    for old_idx, new_idx in reindex_map.items():
        reindexed_mask[array_in == old_idx] = new_idx

    reindexed_mask[reindexed_mask == -1] = ignore_index

    return reindexed_mask

In [88]:
def run_eval(run_name_template,
             model_ckpt_dir,
             states_to_eval, 
             loss_to_eval_options,
             model_kwargs,
             set_this='test',
             compute_r = True,
             include_prior_as_datalayer=False,
             gpu=0):
    
    results_by_state_q = {}
    results_by_state_r = {}
    
    for loss in loss_to_eval_options:
        results_by_state_q[loss] = {}
        results_by_state_r[loss] = {}

        for state in states_to_eval:
            print(state)
            
            run_name = run_name_template.replace('STATE_FILL_IN', state)

            data_dir = f'{torchgeo_data_dir}/enviroatlas_lotp'
            data_dir_this_set = os.path.join(data_dir,f'{state}-{set_this}_tiles-debuffered')

            pred_dir = f'{torchgeo_pred_dir}/{run_name}/enviroatlas_lotp'
            pred_dir_this_set = os.path.join(pred_dir,f'{state}-{set_this}_tiles-debuffered')

            fns = os.listdir(data_dir_this_set)
            tile_ids = np.unique([x[:10] for x in fns])
            print(len(tile_ids))
            
            model_ckpt_fp = os.path.join(model_ckpt_dir, run_name, 'last.ckpt')
            print(model_ckpt_fp)

            accs_q = []
            ious_q = []
            accs_r = []
            ious_r = []
            num_pix = []

            for tile_id in tile_ids:
                fn_this = os.path.join(data_dir_this_set, f'{tile_id}_h_highres_labels.tif')
                image_fn = os.path.join(torchgeo_data_dir, 
                                            'enviroatlas_lotp', 
                                            f'{state}-test_tiles-debuffered',
                                            f'{tile_id}_a_naip.tif')
                print(image_fn)
                t1 = time.time()

                # gather the data
                with rasterio.open(fn_this) as f:
                    hr_lc = f.read()[0]
                # reindex
                hr_lc = reindex_ea(lc.map_raw_lc_to_idx['enviroatlas'][hr_lc], classes_keep)


                
                # prior fns only matter if they're used as model input
                if include_prior_as_datalayer:
                     prior_fns = [image_fn.replace('a_naip',f'prior_from_cooccurrences_101_31')]
                else: 
                    prior_fns = ['']
                                        
                preds_this_soft = run_model.run_through_tiles(model_ckpt_fp,
                                                              input_fns=[image_fn],
                                                              output_fns= [''],
                                                              model_kwargs = model_kwargs,
                                                              gpu = gpu,
                                                              include_prior_as_datalayer=include_prior_as_datalayer,
                                                              prior_fns=[prior_fns],
                                                              return_dont_save=True)

                preds_this = preds_this_soft.argmax(0)
                acc_this_q = (np.array([hr_lc == preds_this])[np.array([hr_lc!=ignore_index])]).mean()

                # ignore 0
                iou_this_q = util.per_class_iou(hr_lc, preds_this, np.arange(0,len(classes_keep)))
                accs_q.append(acc_this_q)
                ious_q.append(iou_this_q)
                num_pix.append((hr_lc != ignore_index).sum())

                if compute_r:
                    # now do r
                    prior_this = rasterio.open(fn_this.replace('h_highres_labels.tif',f'{prior_name}.tif')).read()

                    prior = prior_this / prior_this.sum(axis = 0) + p_add_smooth
                    prior = prior / prior.sum(axis = 0)

                    z = (preds_this_soft.T / preds_this_soft.sum(axis=(1,2)) ).T
                    preds_r = (prior*z).argmax(0)

                    acc_this_r = (np.array([hr_lc == preds_r])[np.array([hr_lc!=ignore_index])]).mean()
                    iou_this_r = util.per_class_iou(hr_lc, preds_r, np.arange(0,len(classes_keep)))

                    accs_r.append(acc_this_r)
                    ious_r.append(iou_this_r)

                t2 = time.time()
                print(f'{t2-t1:.2f} seconds')

            ious_aggregated_q = util.aggregate_ious([x[1] for x in ious_q], [x[2] for x in ious_q])
            acc_aggregated_q = (np.array(accs_q) * np.array(num_pix)).sum() / np.sum(num_pix)
            print(f'For {state} {set_this} set with {loss} loss:')
            print(f'acc q: {acc_aggregated_q}')
            print(f'mean iou q: {np.mean(ious_aggregated_q[0])}')

            if compute_r:
                acc_aggregated_r = (np.array(accs_r) * np.array(num_pix)).sum() / np.sum(num_pix)
                ious_aggregated_r = util.aggregate_ious([x[1] for x in ious_r], [x[2] for x in ious_r])
                print(f'acc r: {acc_aggregated_r}')
                print(f'mean iou r: {np.mean(ious_aggregated_r[0])}')

            print('IoU per class over the tiles (q) is: ')
            print(ious_aggregated_q[0])
            if compute_r:
                print('IoU per class over the tiles (r) is: ')
                print(ious_aggregated_r[0])


            results_by_state_q[loss][state] = {'accs': accs_q,
                                               'ious': ious_q,
                                               'num_pix':num_pix,
                                               'ious_aggregated': ious_aggregated_q,
                                               'acc_aggregated':acc_aggregated_q}

            if compute_r:
                results_by_state_r[loss][state] = {'accs': accs_r,
                                                       'ious': ious_r,
                                                       'num_pix':num_pix,
                                                       'ious_aggregated': ious_aggregated_r,
                                                       'acc_aggregated':acc_aggregated_r}
        

    if compute_r:
        return results_by_state_q, results_by_state_r
    else:
        return results_by_state_q

In [89]:
#for loss in loss_to_eval_options:
states_in_reporting_order = [
    'pittsburgh_pa-2010_1m', 
    'durham_nc-2012_1m',
    'austin_tx-2012_1m',
    'phoenix_az-2010_1m'
]

def print_results(results_by_state, loss_types, identifier=''):
    for loss in loss_types:
        print(loss + " " + identifier)

        result_str = ""
        for state in states_in_reporting_order:

            results_this = results_by_state[loss][state]
            result_str += f"& {np.round(results_this['acc_aggregated']*100,1):.01f} "
            result_str += f"& {np.round(np.mean(results_this['ious_aggregated'][0])*100,1):.01f} "

        print(result_str)
    

# run the evaluation for Q and R

Uncomment one of 1-5 below to evaluate the corresponding experiment.


In [90]:
states_to_eval = [
    'pittsburgh_pa-2010_1m', 
    'durham_nc-2012_1m',
    'austin_tx-2012_1m',
    'phoenix_az-2010_1m'
]

In [91]:
#1. HR model with just NAIP as input
run_name = 'pittsburgh_pa-2010_1m_fcn_0.001_nll'  
model_ckpt_dir = '../output_rep/hp_gridsearch_pittsburgh'

loss_to_eval_options = ['nll']

model_kwargs = {'in_channels': 4, 
                'classes': 5, 
                'output_smooth':1e-8,
                'num_filters': 128}

results_by_state_q_hr = run_eval(run_name,
                                 model_ckpt_dir,
                                 states_to_eval, 
                                 loss_to_eval_options,
                                 model_kwargs=model_kwargs,
                                 set_this='test',
                                 compute_r = False,
                                 gpu=2)

pittsburgh_pa-2010_1m
10
../output_rep/hp_gridsearch_pittsburgh/pittsburgh_pa-2010_1m_fcn_0.001_nll/last.ckpt
/datadrive/esther/torchgeo_data/enviroatlas_lotp/pittsburgh_pa-2010_1m-test_tiles-debuffered/4007925_nw_a_naip.tif
0 of 1
6.93 seconds
/datadrive/esther/torchgeo_data/enviroatlas_lotp/pittsburgh_pa-2010_1m-test_tiles-debuffered/4007926_sw_a_naip.tif
0 of 1
6.77 seconds
/datadrive/esther/torchgeo_data/enviroatlas_lotp/pittsburgh_pa-2010_1m-test_tiles-debuffered/4007933_nw_a_naip.tif
0 of 1
6.75 seconds
/datadrive/esther/torchgeo_data/enviroatlas_lotp/pittsburgh_pa-2010_1m-test_tiles-debuffered/4007933_sw_a_naip.tif
0 of 1
6.75 seconds
/datadrive/esther/torchgeo_data/enviroatlas_lotp/pittsburgh_pa-2010_1m-test_tiles-debuffered/4007934_nw_a_naip.tif
0 of 1
6.92 seconds
/datadrive/esther/torchgeo_data/enviroatlas_lotp/pittsburgh_pa-2010_1m-test_tiles-debuffered/4008031_se_a_naip.tif
0 of 1
6.77 seconds
/datadrive/esther/torchgeo_data/enviroatlas_lotp/pittsburgh_pa-2010_1m-test_tile

In [92]:
results_by_state_q_hr = results_by_state_q
print_results(results_by_state_q_hr, ['nll'], identifier='HR only')

nll HR only
& 89.3 & 69.3 & 74.2 & 35.9 & 71.9 & 36.8 & 6.7 & 13.4 


In [59]:
#2. HR model with just NAIP + NLCD as input
run_name = 'pittsburgh_pa-2010_1m_fcn_0.001_nll_with_prior'  
model_ckpt_dir = '../output_rep/hp_gridsearch_pittsburgh_with_prior_as_input'

loss_to_eval_options = ['nll']

model_kwargs = {'in_channels': 9, 
                'classes': 5, 
                'output_smooth':1e-8,
                'num_filters': 128}

results_by_state_hr_with_prior_as_input = run_eval(run_name,
                                                 model_ckpt_dir,
                                                 states_to_eval, 
                                                 loss_to_eval_options,
                                                 model_kwargs=model_kwargs,
                                                 set_this='test',
                                                 compute_r = False,
                                                   include_prior_as_datalayer=True,
                                                     gpu=2)

10
/datadrive/esther/torchgeo_data/enviroatlas_lotp/pittsburgh_pa-2010_1m-test_tiles-debuffered/4007925_nw_a_naip.tif
['/datadrive/esther/torchgeo_data/enviroatlas_lotp/pittsburgh_pa-2010_1m-test_tiles-debuffered/4007925_nw_prior_from_cooccurrences_101_31.tif']
0 of 1
7.78 seconds
/datadrive/esther/torchgeo_data/enviroatlas_lotp/pittsburgh_pa-2010_1m-test_tiles-debuffered/4007926_sw_a_naip.tif
['/datadrive/esther/torchgeo_data/enviroatlas_lotp/pittsburgh_pa-2010_1m-test_tiles-debuffered/4007926_sw_prior_from_cooccurrences_101_31.tif']
0 of 1
7.51 seconds
/datadrive/esther/torchgeo_data/enviroatlas_lotp/pittsburgh_pa-2010_1m-test_tiles-debuffered/4007933_nw_a_naip.tif
['/datadrive/esther/torchgeo_data/enviroatlas_lotp/pittsburgh_pa-2010_1m-test_tiles-debuffered/4007933_nw_prior_from_cooccurrences_101_31.tif']
0 of 1
7.35 seconds
/datadrive/esther/torchgeo_data/enviroatlas_lotp/pittsburgh_pa-2010_1m-test_tiles-debuffered/4007933_sw_a_naip.tif
['/datadrive/esther/torchgeo_data/enviroatlas

['/datadrive/esther/torchgeo_data/enviroatlas_lotp/austin_tx-2012_1m-test_tiles-debuffered/3009750_se_prior_from_cooccurrences_101_31.tif']
0 of 1
8.32 seconds
/datadrive/esther/torchgeo_data/enviroatlas_lotp/austin_tx-2012_1m-test_tiles-debuffered/3009751_nw_a_naip.tif
['/datadrive/esther/torchgeo_data/enviroatlas_lotp/austin_tx-2012_1m-test_tiles-debuffered/3009751_nw_prior_from_cooccurrences_101_31.tif']
0 of 1
8.31 seconds
For austin_tx-2012_1m test set with nll loss:
acc q: 0.7716083563815472
mean iou q: 0.505376384442352
IoU per class over the tiles (q) is: 
[0.6809374289597171, 0.6421630374301028, 0.06811564440951431, 0.7685540946433391, 0.3671117167690861]
10
/datadrive/esther/torchgeo_data/enviroatlas_lotp/phoenix_az-2010_1m-test_tiles-debuffered/3311117_sw_a_naip.tif
['/datadrive/esther/torchgeo_data/enviroatlas_lotp/phoenix_az-2010_1m-test_tiles-debuffered/3311117_sw_prior_from_cooccurrences_101_31.tif']
0 of 1
8.08 seconds
/datadrive/esther/torchgeo_data/enviroatlas_lotp/ph

In [60]:
print_results(results_by_state_hr_with_prior_as_input, ['nll'], identifier='HR with prior')

nll HR with prior
& 89.5 & 70.5 & 78.9 & 47.9 & 77.2 & 50.5 & 62.8 & 24.2 


In [77]:
#2. QR & RQ model from random initialization
model_ckpt_dir = '../output_rep/ea_qr_from_scratch_rep'

model_kwargs = {'in_channels': 4, 
                'classes': 5, 
                'output_smooth':1e-4,
                'num_filters': 128}

run_name = f'STATE_FILL_IN_fcn_1e-05_qr_forward_{prior_version}_additive_smooth_0.0001_prior_smooth_0.0001'
loss_to_eval_options = ['qr_forward']

results_qr_random_q, results_qr_random_r = run_eval(run_name,
                                                 model_ckpt_dir,
                                                 states_to_eval, 
                                                 loss_to_eval_options,
                                                 model_kwargs=model_kwargs,
                                                 set_this='test',
                                                 compute_r = True,
                                                 gpu=2)

run_name = f'STATE_FILL_IN_fcn_0.001_qr_reverse_{prior_version}_additive_smooth_0.0001_prior_smooth_0.0001'
loss_to_eval_options = ['qr_reverse']

results_rq_random_q, results_rq_random_r = run_eval(run_name,
                                                 model_ckpt_dir,
                                                 states_to_eval, 
                                                 loss_to_eval_options,
                                                 model_kwargs=model_kwargs,
                                                 set_this='test',
                                                 compute_r = True,
                                                 gpu=2)

pittsburgh_pa-2010_1m
10
../output_rep/ea_qr_from_scratch_rep/pittsburgh_pa-2010_1m_fcn_1e-05_qr_forward_from_cooccurrences_101_31_additive_smooth_0.0001_prior_smooth_0.0001/last.ckpt
/datadrive/esther/torchgeo_data/enviroatlas_lotp/pittsburgh_pa-2010_1m-test_tiles-debuffered/4007925_nw_a_naip.tif
0 of 1
11.76 seconds
/datadrive/esther/torchgeo_data/enviroatlas_lotp/pittsburgh_pa-2010_1m-test_tiles-debuffered/4007926_sw_a_naip.tif
0 of 1
11.83 seconds
/datadrive/esther/torchgeo_data/enviroatlas_lotp/pittsburgh_pa-2010_1m-test_tiles-debuffered/4007933_nw_a_naip.tif
0 of 1
11.96 seconds
/datadrive/esther/torchgeo_data/enviroatlas_lotp/pittsburgh_pa-2010_1m-test_tiles-debuffered/4007933_sw_a_naip.tif
0 of 1
11.91 seconds
/datadrive/esther/torchgeo_data/enviroatlas_lotp/pittsburgh_pa-2010_1m-test_tiles-debuffered/4007934_nw_a_naip.tif
0 of 1
11.94 seconds
/datadrive/esther/torchgeo_data/enviroatlas_lotp/pittsburgh_pa-2010_1m-test_tiles-debuffered/4008031_se_a_naip.tif
0 of 1
11.78 seconds


0 of 1
11.54 seconds
/datadrive/esther/torchgeo_data/enviroatlas_lotp/pittsburgh_pa-2010_1m-test_tiles-debuffered/4007933_nw_a_naip.tif
0 of 1
11.68 seconds
/datadrive/esther/torchgeo_data/enviroatlas_lotp/pittsburgh_pa-2010_1m-test_tiles-debuffered/4007933_sw_a_naip.tif
0 of 1
11.71 seconds
/datadrive/esther/torchgeo_data/enviroatlas_lotp/pittsburgh_pa-2010_1m-test_tiles-debuffered/4007934_nw_a_naip.tif
0 of 1
11.74 seconds
/datadrive/esther/torchgeo_data/enviroatlas_lotp/pittsburgh_pa-2010_1m-test_tiles-debuffered/4008031_se_a_naip.tif
0 of 1
11.57 seconds
/datadrive/esther/torchgeo_data/enviroatlas_lotp/pittsburgh_pa-2010_1m-test_tiles-debuffered/4008032_nw_a_naip.tif
0 of 1
11.64 seconds
/datadrive/esther/torchgeo_data/enviroatlas_lotp/pittsburgh_pa-2010_1m-test_tiles-debuffered/4008032_se_a_naip.tif
0 of 1
11.80 seconds
/datadrive/esther/torchgeo_data/enviroatlas_lotp/pittsburgh_pa-2010_1m-test_tiles-debuffered/4008048_ne_a_naip.tif
0 of 1
11.79 seconds
/datadrive/esther/torchgeo_

In [78]:
print_results(results_qr_random_q, ['qr_forward'], identifier='(q) from random')
print_results(results_qr_random_r, ['qr_forward'], identifier='(r) from random')
print_results(results_rq_random_q, ['qr_reverse'], identifier='(q) from random')
print_results(results_rq_random_r, ['qr_reverse'], identifier='(r) from random')

qr_forward (q) from random
& 80.5 & 56.8 & 78.3 & 44.4 & 79.2 & 50.5 & 75.2 & 29.5 
qr_forward (r) from random
& 80.7 & 57.5 & 78.5 & 46.4 & 79.7 & 52.0 & 75.9 & 33.8 
qr_reverse (q) from random
& 77.6 & 53.3 & 65.8 & 23.3 & 73.8 & 43.0 & 61.8 & 18.6 
qr_reverse (r) from random
& 77.6 & 53.3 & 65.8 & 23.3 & 73.8 & 43.1 & 61.8 & 18.6 


In [80]:
#3. QR & RQ model from pittsburch initialization
model_ckpt_dir = '../output_rep/ea_qr_from_pittsburgh_model_rep'

model_kwargs = {'in_channels': 4, 
                'classes': 5, 
                'output_smooth':1e-4,
                'num_filters': 128}

run_name = f'pa_checkpoint_STATE_FILL_IN_fcn_1e-05_qr_forward_{prior_version}_additive_smooth_0.0001_prior_smooth_0.0001'
loss_to_eval_options = ['qr_forward']

results_qr_ckpt_q, results_qr_ckpt_r = run_eval(run_name,
                                                 model_ckpt_dir,
                                                 states_to_eval, 
                                                 loss_to_eval_options,
                                                 model_kwargs=model_kwargs,
                                                 set_this='test',
                                                 compute_r = True,
                                                 gpu=2)

run_name = f'pa_checkpoint_STATE_FILL_IN_fcn_0.001_qr_reverse_{prior_version}_additive_smooth_0.0001_prior_smooth_0.0001'
loss_to_eval_options = ['qr_reverse']

results_rq_ckpt_q, results_rq_ckpt_r = run_eval(run_name,
                                                 model_ckpt_dir,
                                                 states_to_eval, 
                                                 loss_to_eval_options,
                                                 model_kwargs=model_kwargs,
                                                 set_this='test',
                                                 compute_r = True,
                                                 gpu=2)

pittsburgh_pa-2010_1m
10
../output_rep/ea_qr_from_pittsburgh_model_rep/pa_checkpoint_pittsburgh_pa-2010_1m_fcn_1e-05_qr_forward_from_cooccurrences_101_31_additive_smooth_0.0001_prior_smooth_0.0001/last.ckpt
/datadrive/esther/torchgeo_data/enviroatlas_lotp/pittsburgh_pa-2010_1m-test_tiles-debuffered/4007925_nw_a_naip.tif
0 of 1
13.13 seconds
/datadrive/esther/torchgeo_data/enviroatlas_lotp/pittsburgh_pa-2010_1m-test_tiles-debuffered/4007926_sw_a_naip.tif
0 of 1
12.79 seconds
/datadrive/esther/torchgeo_data/enviroatlas_lotp/pittsburgh_pa-2010_1m-test_tiles-debuffered/4007933_nw_a_naip.tif
0 of 1
12.87 seconds
/datadrive/esther/torchgeo_data/enviroatlas_lotp/pittsburgh_pa-2010_1m-test_tiles-debuffered/4007933_sw_a_naip.tif
0 of 1
12.23 seconds
/datadrive/esther/torchgeo_data/enviroatlas_lotp/pittsburgh_pa-2010_1m-test_tiles-debuffered/4007934_nw_a_naip.tif
0 of 1
12.73 seconds
/datadrive/esther/torchgeo_data/enviroatlas_lotp/pittsburgh_pa-2010_1m-test_tiles-debuffered/4008031_se_a_naip.ti

0 of 1
12.24 seconds
/datadrive/esther/torchgeo_data/enviroatlas_lotp/pittsburgh_pa-2010_1m-test_tiles-debuffered/4007926_sw_a_naip.tif
0 of 1
11.93 seconds
/datadrive/esther/torchgeo_data/enviroatlas_lotp/pittsburgh_pa-2010_1m-test_tiles-debuffered/4007933_nw_a_naip.tif
0 of 1
12.04 seconds
/datadrive/esther/torchgeo_data/enviroatlas_lotp/pittsburgh_pa-2010_1m-test_tiles-debuffered/4007933_sw_a_naip.tif
0 of 1
12.15 seconds
/datadrive/esther/torchgeo_data/enviroatlas_lotp/pittsburgh_pa-2010_1m-test_tiles-debuffered/4007934_nw_a_naip.tif
0 of 1
12.04 seconds
/datadrive/esther/torchgeo_data/enviroatlas_lotp/pittsburgh_pa-2010_1m-test_tiles-debuffered/4008031_se_a_naip.tif
0 of 1
11.89 seconds
/datadrive/esther/torchgeo_data/enviroatlas_lotp/pittsburgh_pa-2010_1m-test_tiles-debuffered/4008032_nw_a_naip.tif
0 of 1
12.10 seconds
/datadrive/esther/torchgeo_data/enviroatlas_lotp/pittsburgh_pa-2010_1m-test_tiles-debuffered/4008032_se_a_naip.tif
0 of 1
12.24 seconds
/datadrive/esther/torchgeo_

In [81]:
print_results(results_qr_ckpt_q, ['qr_forward'], identifier='(q) from ckpt')
print_results(results_qr_ckpt_r, ['qr_forward'], identifier='(r) from ckpt')
print_results(results_rq_ckpt_q, ['qr_reverse'], identifier='(q) from ckpt')
print_results(results_rq_ckpt_r, ['qr_reverse'], identifier='(r) from ckpt')

qr_forward (q) from ckpt
& 80.6 & 58.5 & 78.9 & 47.7 & 76.6 & 49.1 & 75.8 & 45.4 
qr_forward (r) from ckpt
& 80.6 & 58.7 & 79.0 & 48.4 & 76.6 & 49.5 & 76.2 & 46.0 
qr_reverse (q) from ckpt
& 84.3 & 59.6 & 75.6 & 28.6 & 76.5 & 47.5 & 63.7 & 19.5 
qr_reverse (r) from ckpt
& 84.3 & 59.6 & 75.4 & 31.5 & 76.5 & 47.5 & 63.7 & 19.5 


In [83]:
#3. QR & RQ model from pittsburch initialization
model_ckpt_dir = '../output_rep/ea_qr_learned_prior'

model_kwargs = {'in_channels': 4, 
                'classes': 5, 
                'output_smooth':1e-4,
                'num_filters': 128}

run_name = f'pa_checkpoint_STATE_FILL_IN_fcn_1e-05_qr_forward_learned_101_31_additive_smooth_0.0001_prior_smooth_0.0001'
loss_to_eval_options = ['qr_forward']

results_qr_from_learned_q, results_qr_from_learned_r = run_eval(run_name,
                                                 model_ckpt_dir,
                                                 states_to_eval, 
                                                 loss_to_eval_options,
                                                 model_kwargs=model_kwargs,
                                                 set_this='test',
                                                 compute_r = True,
                                                 gpu=2)


pittsburgh_pa-2010_1m
10
../output_rep/ea_qr_learned_prior/pa_checkpoint_pittsburgh_pa-2010_1m_fcn_1e-05_qr_forward_learned_101_31_additive_smooth_0.0001_prior_smooth_0.0001/last.ckpt
/datadrive/esther/torchgeo_data/enviroatlas_lotp/pittsburgh_pa-2010_1m-test_tiles-debuffered/4007925_nw_a_naip.tif
0 of 1
12.22 seconds
/datadrive/esther/torchgeo_data/enviroatlas_lotp/pittsburgh_pa-2010_1m-test_tiles-debuffered/4007926_sw_a_naip.tif
0 of 1
12.14 seconds
/datadrive/esther/torchgeo_data/enviroatlas_lotp/pittsburgh_pa-2010_1m-test_tiles-debuffered/4007933_nw_a_naip.tif
0 of 1
12.03 seconds
/datadrive/esther/torchgeo_data/enviroatlas_lotp/pittsburgh_pa-2010_1m-test_tiles-debuffered/4007933_sw_a_naip.tif
0 of 1
12.08 seconds
/datadrive/esther/torchgeo_data/enviroatlas_lotp/pittsburgh_pa-2010_1m-test_tiles-debuffered/4007934_nw_a_naip.tif
0 of 1
12.21 seconds
/datadrive/esther/torchgeo_data/enviroatlas_lotp/pittsburgh_pa-2010_1m-test_tiles-debuffered/4008031_se_a_naip.tif
0 of 1
11.86 seconds


In [84]:
print_results(results_qr_from_learned_q, ['qr_forward'], identifier='(q) from learned')
print_results(results_qr_from_learned_r, ['qr_forward'], identifier='(r) from learned')


qr_forward (q) from learned
& 82.4 & 63.7 & 79.0 & 48.7 & 79.4 & 51.3 & 73.4 & 42.8 
qr_forward (r) from learned
& 82.4 & 64.0 & 79.2 & 49.5 & 79.1 & 51.9 & 73.6 & 43.1 
